# Estimation for Rigid Transformation
+ Formalization
$$
q_i = R_{xyz} p_i + t + \epsilon, \epsilon \sim N(0,1),
$$
+ $t$ and $R_{xyz}$ is targets to estimate and $R_{xyz} = R_x(\theta_x) R_y(\theta_y) R_z(\theta_z)$.
+ Optimizing each parameters $t$, $R_x(\theta_x)$, $R_y(\theta_y)$, and $R_z(\theta_z)$ are considered here.
+ First, this notebook consider about $t$ and $R_x(\theta_x)$

In [1]:
import numpy as np
import scipy

In [2]:
def rot_x(theta_x: float) -> np.ndarray:
    ct = np.cos(theta_x)
    st = np.sin(theta_x)
    return np.array([
        [1, 0, 0],
        [0, ct, -st],
        [0, st, ct]
    ])
    pass

def rot_y(theta_y: float) -> np.ndarray:
    ct = np.cos(theta_y)
    st = np.sin(theta_y)
    return np.array([
        [ct, 0, -st],
        [0, 1, 0],
        [st, 0, ct]
    ])

def rot_z(theta_z: float) -> np.ndarray:
    ct = np.cos(theta_z)
    st = np.sin(theta_z)
    return np.array([
        [ct, -st, 0],
        [st, ct, 0],
        [0, 0, 1],
    ])

# Loop to estimate R_x, R_y, R_z, and t with a covariance noise

In [80]:
n_sample = 1000
n_test = 10000
data_seed = 20240630
np.random.seed(data_seed)

In [81]:
scale = np.array([
    [1, 0.5, 0.1],
    [0.5, 1, 0.8],
    [0.1, 0.8, 1],
])

In [82]:
cov = scipy.stats.wishart.rvs(df=10, scale=scale)
sq_cov = scipy.linalg.sqrtm(cov)

# cov = np.eye(3)

In [83]:
true_theta_x = np.random.uniform(low=0, high=2*np.pi)
true_theta_y = np.random.uniform(low=0, high=2*np.pi)
true_theta_z = np.random.uniform(low=0, high=2*np.pi)
true_trans_vec = np.random.normal(size=3)
true_rot = rot_x(true_theta_x) @ rot_y(true_theta_y) @ rot_z(true_theta_z)

train_noise = np.random.normal(size=(n_sample,3))
test_noise = np.random.normal(size=(n_test,3))
# np.random.multivariate_normal(mean=np.zeros(3), cov=cov, size=n_sample)
# test_noise = np.random.multivariate_normal(mean=np.zeros(3), cov=cov, size=n_test)

p_i = np.random.normal(size=(n_sample, 3))
p_i_dash = np.linalg.solve(sq_cov, p_i.T).T
# p_i_dash = p_i @ sq_cov.T
q_i = p_i @ true_rot.T + true_trans_vec + train_noise @ sq_cov.T
q_i_dash = np.linalg.solve(sq_cov, q_i.T).T
# q_i_dash = q_i_dash = p_i_dash @ true_rot.T + true_trans_vec + train_noise @ sq_cov.T

test_p_i = np.random.normal(size=(n_test, 3))
test_p_i_dash = np.linalg.solve(sq_cov, test_p_i.T).T
# test_p_i_dash = test_p_i @ sq_cov.T
test_q_i = test_p_i @ true_rot.T + true_trans_vec + test_noise @ sq_cov.T
test_q_i_dash = test_p_i_dash @ true_rot.T + true_trans_vec + test_noise @ sq_cov.T

# np.random.normal(scale=1, size=(n_sample, 3))

In [84]:
# p_i_dash = np.linalg.solve(sq_cov, p_i.T).T
# test_p_i_dash = np.linalg.solve(sq_cov, test_p_i.T).T

# q_i_dash = p_i_dash @ true_rot.T + true_trans_vec + np.random.multivariate_normal(mean=np.zeros(3), cov=cov, size=n_sample)

In [85]:
def estimate_theta(sin_coef: float, cos_coef: float) -> float:
    """ estimate maximizing theta of sin_coef sin(theta) + cos_coef cos(theta),
    where theta in [0,2pi].
    consider the 1st and 2nd order derivative, the value is obtained.
    """
    
    dist = np.sqrt(sin_coef**2 + cos_coef**2)
    if sin_coef >= 0:
        delta = np.arcsin(cos_coef/dist)
        pass
    else:
        delta = np.pi - np.arcsin(cos_coef/dist)
        pass
    candidate_est_theta = np.arange(4) * np.pi - delta
    est_theta = min(
        filter(
            lambda theta: 
            0 <= theta and theta <= 2*np.pi # for range of theta
            and np.cos(theta + delta) < 0, # for 2nd order derivative is negative
            candidate_est_theta,
        )
    )
    return est_theta
    pass

In [86]:
n_ite = 100
ln_seed = 20240701

In [87]:
def estimate_theta_x(x_i: np.ndarray, y_i: np.ndarray, theta_y: float, theta_z: float, trans_vec: np.ndarray) -> float:
    tilde_y_i = y_i - trans_vec
    tilde_x_i = x_i @ rot_z(theta_z).T @ rot_y(theta_y).T    
    sin_coef = -(tilde_x_i[:,1]*tilde_y_i[:,1] + tilde_x_i[:,2]*tilde_y_i[:,2]).sum()
    cos_coef = (-tilde_x_i[:,2]*tilde_y_i[:,1] + tilde_x_i[:,1]*tilde_y_i[:,2]).sum()
    return estimate_theta(sin_coef=sin_coef, cos_coef=cos_coef)

def estimate_theta_y(x_i: np.ndarray, y_i: np.ndarray, theta_x: float, theta_z: float, trans_vec: np.ndarray) -> float:
    tilde_y_i = (y_i - trans_vec) @ rot_x(theta_x)
    tilde_x_i = x_i @ rot_z(theta_z).T
    sin_coef = -(tilde_x_i[:,0]*tilde_y_i[:,0] + tilde_x_i[:,2]*tilde_y_i[:,2]).sum()
    cos_coef = (-tilde_x_i[:,2]*tilde_y_i[:,0] + tilde_x_i[:,0]*tilde_y_i[:,2]).sum()
    return estimate_theta(sin_coef=sin_coef, cos_coef=cos_coef)

def estimate_theta_z(x_i: np.ndarray, y_i: np.ndarray, theta_x: float, theta_y: float, trans_vec: np.ndarray) -> float:
    tilde_y_i = (y_i - trans_vec) @ rot_x(theta_x) @ rot_y(theta_y)
    tilde_x_i = x_i
    sin_coef = -(tilde_x_i[:,0]*tilde_y_i[:,0] + tilde_x_i[:,1]*tilde_y_i[:,1]).sum()
    cos_coef = (-tilde_x_i[:,1]*tilde_y_i[:,0] + tilde_x_i[:,0]*tilde_y_i[:,1]).sum()    
    return estimate_theta(sin_coef=sin_coef, cos_coef=cos_coef)

def estimate_trans_vec(x_i: np.ndarray, y_i: np.ndarray, theta_x: float, theta_y: float, theta_z: float) -> np.ndarray:
    est_rot = rot_x(theta_x) @ rot_y(theta_y) @ rot_z(theta_z)
    return (y_i - x_i @ est_rot.T).mean(axis=0)

In [88]:
train_loglik = []
test_loglik = []
np.random.seed(ln_seed)

# initialize
est_theta_x = np.random.uniform(low=0, high=2*np.pi)
est_theta_y = np.random.uniform(low=0, high=2*np.pi)
est_theta_z = np.random.uniform(low=0, high=2*np.pi)
est_trans_vec = np.random.normal(size=3)

for _ in range(n_ite):
    # estimate theta_z
    est_theta_z = estimate_theta_z(x_i=p_i, y_i=q_i, theta_x=est_theta_x, theta_y=est_theta_y, trans_vec=est_trans_vec)
    # est_theta_z = estimate_theta_z(x_i=p_i, y_i=q_i, theta_x=est_theta_x, theta_y=est_theta_y, trans_vec=est_trans_vec)
    
    # estimate theta_y
    est_theta_y = estimate_theta_y(x_i=p_i, y_i=q_i, theta_x=est_theta_x, theta_z=est_theta_z, trans_vec=est_trans_vec)
    # est_theta_y = estimate_theta_y(x_i=p_i, y_i=q_i, theta_x=est_theta_x, theta_z=est_theta_z, trans_vec=est_trans_vec)
    
    # estimate theta_x
    est_theta_x = estimate_theta_x(x_i=p_i, y_i=q_i, theta_y=est_theta_y, theta_z=est_theta_z, trans_vec=est_trans_vec)
    # est_theta_x = estimate_theta_x(x_i=p_i, y_i=q_i, theta_y=est_theta_y, theta_z=est_theta_z, trans_vec=est_trans_vec)

    # estimate translation vector
    est_trans_vec = estimate_trans_vec(x_i=p_i, y_i=q_i, theta_x=est_theta_x, theta_y=est_theta_y, theta_z=est_theta_z)
    # est_trans_vec = estimate_trans_vec(x_i=p_i, y_i=q_i, theta_x=est_theta_x, theta_y=est_theta_y, theta_z=est_theta_z)
    
    est_rot = rot_x(est_theta_x) @ rot_y(est_theta_y) @ rot_z(est_theta_z)
    train_loglik.append(-((q_i - (p_i @ est_rot.T + est_trans_vec))**2).sum(axis=1).sum()/2)
    test_loglik.append(-((test_q_i - (test_p_i @ est_rot.T + est_trans_vec))**2).sum(axis=1).sum()/2)
    # test_res = test_q_i - (test_p_i @ est_rot.T + est_trans_vec)
    # test_loglik.append(-((test_res * np.linalg.solve(cov, test_res.T).T)**2).sum(axis=1).sum()/2)
    

In [89]:
np.linalg.solve(cov, (test_p_i @ est_rot.T + est_trans_vec).T).T

array([[-0.50048746,  1.16108848, -1.33158083],
       [-0.08867117,  0.33696559, -0.57193993],
       [-0.46161828,  1.13011273, -1.21780028],
       ...,
       [-0.053953  ,  0.2047059 , -0.25716161],
       [-0.21510446,  0.85970422, -1.1046678 ],
       [-0.23027783,  0.47492961, -0.68186508]])

In [90]:
train_loglik[:10], test_loglik[:10]

([-19581.73701101075,
  -19578.568180585404,
  -19578.554032835025,
  -19578.55402917855,
  -19578.55402917762,
  -19578.55402917762,
  -19578.55402917762,
  -19578.55402917762,
  -19578.55402917762,
  -19578.55402917762],
 [-196223.0868967063,
  -196196.53465251153,
  -196198.5418353558,
  -196198.59090393357,
  -196198.59179635008,
  -196198.591809031,
  -196198.59180916747,
  -196198.59180916817,
  -196198.59180916817,
  -196198.59180916817])

In [91]:
(est_theta_x, est_theta_y, est_theta_z), (true_theta_x, true_theta_y, true_theta_z)

((3.8500985902986082, 2.9812548743263596, 5.237219391899544),
 (0.7390459478398709, 0.09773281292055837, 2.1352211603377387))

In [92]:
est_rot.T @ true_rot

array([[ 0.99766375, -0.03671538, -0.05761101],
       [ 0.03450808,  0.99864894, -0.03885222],
       [ 0.05895965,  0.0367734 ,  0.99758282]])

In [93]:
est_rot, true_rot

(array([[-0.49463928, -0.85430869, -0.15965166],
        [ 0.70918865, -0.2905738 , -0.64235374],
        [ 0.50237779, -0.43095654,  0.74959523]]),
 array([[-0.53237724, -0.84086453, -0.0975773 ],
        [ 0.65963171, -0.33984088, -0.67036869],
        [ 0.5305285 , -0.42125411,  0.73558445]]))

In [94]:
np.sqrt(((est_trans_vec - true_trans_vec)**2).sum())

0.31328851146130243

In [95]:
train_loglik = []
test_loglik = []
np.random.seed(ln_seed)

# initialize
est_theta_x = np.random.uniform(low=0, high=2*np.pi)
est_theta_y = np.random.uniform(low=0, high=2*np.pi)
est_theta_z = np.random.uniform(low=0, high=2*np.pi)
est_trans_vec = np.random.normal(size=3)

for _ in range(n_ite):
    # estimate theta_z
    est_theta_z = estimate_theta_z(x_i=p_i_dash, y_i=q_i, theta_x=est_theta_x, theta_y=est_theta_y, trans_vec=est_trans_vec)
    # est_theta_z = estimate_theta_z(x_i=p_i, y_i=q_i, theta_x=est_theta_x, theta_y=est_theta_y, trans_vec=est_trans_vec)
    
    # estimate theta_y
    est_theta_y = estimate_theta_y(x_i=p_i_dash, y_i=q_i, theta_x=est_theta_x, theta_z=est_theta_z, trans_vec=est_trans_vec)
    # est_theta_y = estimate_theta_y(x_i=p_i, y_i=q_i, theta_x=est_theta_x, theta_z=est_theta_z, trans_vec=est_trans_vec)
    
    # estimate theta_x
    est_theta_x = estimate_theta_x(x_i=p_i_dash, y_i=q_i, theta_y=est_theta_y, theta_z=est_theta_z, trans_vec=est_trans_vec)
    # est_theta_x = estimate_theta_x(x_i=p_i, y_i=q_i, theta_y=est_theta_y, theta_z=est_theta_z, trans_vec=est_trans_vec)

    # estimate translation vector
    est_trans_vec = estimate_trans_vec(x_i=p_i_dash, y_i=q_i, theta_x=est_theta_x, theta_y=est_theta_y, theta_z=est_theta_z)
    # est_trans_vec = estimate_trans_vec(x_i=p_i, y_i=q_i, theta_x=est_theta_x, theta_y=est_theta_y, theta_z=est_theta_z)
    
    est_rot = rot_x(est_theta_x) @ rot_y(est_theta_y) @ rot_z(est_theta_z)
    train_loglik.append(-((q_i - (p_i_dash @ est_rot.T + est_trans_vec))**2).sum(axis=1).sum()/2)
    test_loglik.append(-((test_q_i - (test_p_i_dash @ est_rot.T + est_trans_vec))**2).sum(axis=1).sum()/2)
    # test_res = test_q_i - (test_p_i @ est_rot.T + est_trans_vec)
    # test_loglik.append(-((test_res * np.linalg.solve(cov, test_res.T).T)**2).sum(axis=1).sum()/2)


In [96]:
train_loglik[:10], test_loglik[:10]

([-20424.61476106367,
  -20415.44512767668,
  -20414.685449419492,
  -20414.6130036549,
  -20414.6064210166,
  -20414.605834878617,
  -20414.605782961513,
  -20414.605778370926,
  -20414.605777965215,
  -20414.605777929366],
 [-202606.22838279532,
  -202507.92015856295,
  -202492.9422163909,
  -202489.6761891549,
  -202488.81555450693,
  -202488.57011761784,
  -202488.49807278498,
  -202488.47673887448,
  -202488.47040447558,
  -202488.46852219073])

In [97]:
(est_theta_x, est_theta_y, est_theta_z), (true_theta_x, true_theta_y, true_theta_z)

((3.7952088443357317, 3.019237399458046, 5.221838580371456),
 (0.7390459478398709, 0.09773281292055837, 2.1352211603377387))

In [98]:
est_rot.T @ true_rot

array([[ 0.99530591, -0.04452918, -0.08592617],
       [ 0.04643489,  0.99871487,  0.02030777],
       [ 0.08491145, -0.02420241,  0.99609452]])

In [99]:
est_rot, true_rot

(array([[-0.48405076, -0.86648636, -0.12205019],
        [ 0.72927038, -0.3223879 , -0.60351532],
        [ 0.48359029, -0.38113964,  0.78795495]]),
 array([[-0.53237724, -0.84086453, -0.0975773 ],
        [ 0.65963171, -0.33984088, -0.67036869],
        [ 0.5305285 , -0.42125411,  0.73558445]]))

In [100]:
est_trans_vec, true_trans_vec

(array([-0.01174337, -0.69702611, -1.15265375]),
 array([ 0.325998  , -0.73163138, -1.26774885]))

In [101]:
np.sqrt(((est_trans_vec - true_trans_vec)**2).sum())

0.35848799099798717

In [102]:
train_loglik = []
test_loglik = []
np.random.seed(ln_seed)

# initialize
est_theta_x = np.random.uniform(low=0, high=2*np.pi)
est_theta_y = np.random.uniform(low=0, high=2*np.pi)
est_theta_z = np.random.uniform(low=0, high=2*np.pi)
est_trans_vec = np.random.normal(size=3)

for _ in range(n_ite):
    # estimate theta_z
    est_theta_z = estimate_theta_z(x_i=p_i, y_i=q_i_dash, theta_x=est_theta_x, theta_y=est_theta_y, trans_vec=est_trans_vec)
    # est_theta_z = estimate_theta_z(x_i=p_i, y_i=q_i, theta_x=est_theta_x, theta_y=est_theta_y, trans_vec=est_trans_vec)
    
    # estimate theta_y
    est_theta_y = estimate_theta_y(x_i=p_i, y_i=q_i_dash, theta_x=est_theta_x, theta_z=est_theta_z, trans_vec=est_trans_vec)
    # est_theta_y = estimate_theta_y(x_i=p_i, y_i=q_i, theta_x=est_theta_x, theta_z=est_theta_z, trans_vec=est_trans_vec)
    
    # estimate theta_x
    est_theta_x = estimate_theta_x(x_i=p_i, y_i=q_i_dash, theta_y=est_theta_y, theta_z=est_theta_z, trans_vec=est_trans_vec)
    # est_theta_x = estimate_theta_x(x_i=p_i, y_i=q_i, theta_y=est_theta_y, theta_z=est_theta_z, trans_vec=est_trans_vec)

    # estimate translation vector
    est_trans_vec = estimate_trans_vec(x_i=p_i, y_i=q_i_dash, theta_x=est_theta_x, theta_y=est_theta_y, theta_z=est_theta_z)
    # est_trans_vec = estimate_trans_vec(x_i=p_i, y_i=q_i, theta_x=est_theta_x, theta_y=est_theta_y, theta_z=est_theta_z)
    
    est_rot = rot_x(est_theta_x) @ rot_y(est_theta_y) @ rot_z(est_theta_z)
    train_loglik.append(-((q_i_dash - (p_i @ est_rot.T + est_trans_vec))**2).sum(axis=1).sum()/2)
    test_loglik.append(-((test_q_i - (test_p_i_dash @ est_rot.T + est_trans_vec))**2).sum(axis=1).sum()/2)

In [103]:
train_loglik[:10], test_loglik[:10]

([-2018.8980945768308,
  -2016.3039280663475,
  -2016.2756457195442,
  -2016.2749798069776,
  -2016.2749644502485,
  -2016.2749640958955,
  -2016.274964087719,
  -2016.2749640875302,
  -2016.2749640875259,
  -2016.2749640875256],
 [-208561.45201103424,
  -208570.09869975803,
  -208573.0499177628,
  -208573.5532766191,
  -208573.630838654,
  -208573.64264674872,
  -208573.6444410577,
  -208573.64471363643,
  -208573.64475504291,
  -208573.64476133278])

In [104]:
(est_theta_x, est_theta_y, est_theta_z), (true_theta_x, true_theta_y, true_theta_z)

((3.8440454081121747, 2.990630600996062, 5.229159760434147),
 (0.7390459478398709, 0.09773281292055837, 2.1352211603377387))

In [105]:
est_rot.T @ true_rot

array([[ 0.99735957, -0.04384354, -0.05789326],
       [ 0.0422671 ,  0.99870885, -0.02818006],
       [ 0.05905402,  0.02565867,  0.99792497]])

In [106]:
est_rot, true_rot

(array([[-0.48845598, -0.85953116, -0.15038931],
        [ 0.71159966, -0.29263034, -0.63874362],
        [ 0.50501157, -0.41901513,  0.75457911]]),
 array([[-0.53237724, -0.84086453, -0.0975773 ],
        [ 0.65963171, -0.33984088, -0.67036869],
        [ 0.5305285 , -0.42125411,  0.73558445]]))

In [107]:
est_trans_vec, true_trans_vec

(array([ 0.0061391 ,  0.04209442, -0.48001622]),
 array([ 0.325998  , -0.73163138, -1.26774885]))

In [79]:
np.sqrt(((est_trans_vec - true_trans_vec)**2).sum())

1.4001985615514452

In [29]:
loglik

[-6326.812401881012,
 -6308.634723421868,
 -6308.40471523716,
 -6308.403234952914,
 -6308.403224972835,
 -6308.403224905259,
 -6308.403224904801,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.403224904798,
 -6308.4032249

In [31]:
est_rot

array([[ 0.96195309,  0.21698008,  0.16602983],
       [-0.15632946,  0.93550722, -0.31683961],
       [-0.22406999,  0.27882949,  0.93383444]])

In [32]:
true_rot

array([[ 0.97001203,  0.19408688,  0.1463111 ],
       [-0.12798579,  0.91961591, -0.37138418],
       [-0.20663082,  0.34152138,  0.91687886]])

### estimated rot mat
+ optimized function is
$$
L(\theta_x) = \sum_i <R_x(\theta_x)p_i, q_i - t>,
$$
where redundant terms to optimize $\theta_x$ are omitted.
By calculating max value of $L(\theta_x)$, estimated $\theta_x$ is obtained.

In [36]:
tilde_q_i = q_i - est_trans_vec
# tilde_q_i = q_i - true_trans_vec
alpha_x = (-p_i[:,2]*tilde_q_i[:,1] + p_i[:,1]*tilde_q_i[:,2]).sum()
beta_x = -(p_i[:,1]*tilde_q_i[:,1] + p_i[:,2]*tilde_q_i[:,2]).sum()

In [37]:
if beta_x >= 0:
    gamma_x = np.arcsin(alpha_x/np.sqrt(alpha_x**2 + beta_x**2))
else:
    gamma_x = np.pi - np.arcsin(alpha_x/np.sqrt(alpha_x**2 + beta_x**2))

In [38]:
candidate_est_theta_x = np.arange(4) * np.pi - gamma_x
candidate_est_theta_x = candidate_est_theta_x[(0<=candidate_est_theta_x) & (candidate_est_theta_x <= 2*np.pi)]
est_theta_x = min(candidate_est_theta_x[np.cos(candidate_est_theta_x + gamma_x) < 0])
# candidate_est_theta_x[np.argmax(alpha_x*np.sin(candidate_est_theta_x) + beta_x*np.cos(candidate_est_theta_x))]

In [39]:
est_rot = rot_x(est_theta_x)
-((q_i - (p_i @ est_rot.T + est_trans_vec))**2).sum(axis=1).sum()/2

-163.38790035034546

In [40]:
true_theta_x

4.247931759760334

In [41]:
est_theta_x

4.262658203214274

In [42]:
est_theta_x

4.262658203214274

In [43]:
[np.cos(candidate_est_theta_x + gamma_x) < 0]

[array([False,  True])]

In [44]:
est_trans_vec, np.rad2deg(est_theta_x), np.rad2deg(true_theta_x)

(array([-1.51060822,  1.47331913,  1.14509845]),
 244.2323245509967,
 243.3885614938479)

In [45]:
rot_x(true_theta_x)

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        , -0.44793759,  0.89406483],
       [ 0.        , -0.89406483, -0.44793759]])

In [46]:
rot_x(est_theta_x)

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        , -0.4347231 ,  0.90056417],
       [ 0.        , -0.90056417, -0.4347231 ]])

In [47]:
est_theta_x

4.262658203214274

In [48]:
np.rad2deg(est_theta_x)

244.2323245509967

In [49]:
np.rad2deg(true_theta_x)

243.3885614938479

In [50]:
est_theta_x

4.262658203214274

In [51]:
est_trans_vec

array([-1.51060822,  1.47331913,  1.14509845])

In [52]:
true_trans_vec

array([-1.33375912,  1.30214425,  1.31281983])

In [53]:
np.sqrt(alpha_x**2 + beta_x**2) * np.sin(est_theta_x + gamma_x)

5.037827037954152e-14

In [54]:
alpha_x * np.cos(est_theta_x) - beta_x * np.sin(est_theta_x)

69.62406943783711

In [55]:
gamma_x = np.arccos(-beta_x / np.sqrt(alpha_x**2 + beta_x**2))
candidate_est_theta_x = np.arange(3) * np.pi - gamma_x
est_theta_x = min(candidate_est_theta_x[(0<=candidate_est_theta_x) & (candidate_est_theta_x <= 2*np.pi)].tolist())

In [56]:
np.rad2deg(true_theta_x)

243.3885614938479

In [57]:
np.rad2deg(est_theta_x)

64.23232455099672